In [1]:
# import the necessary packages
import os
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
from tensorflow.keras.models import load_model


In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1_W2gFFZmy6ZyC8TPlxB49eDFswdBsQqo',
                                    dest_path='content/face_mask_detection.zip',
                                    unzip=True)

In [3]:
# Loading the order of the image's name that has been provided

#test_image_order = pd.read_csv("./face_mask_detection/Testing_set_face_mask.csv")

test_image_order = pd.read_csv("/content/content/face_mask_detection/Testing_set_face_mask.csv")
test_image_order.head()


,filename,label
0,Image_1.jpg,NaN
1,Image_2.jpg,NaN
2,Image_3.jpg,NaN
3,Image_4.jpg,NaN
4,Image_5.jpg,NaN


In [4]:
file_paths = [[fname, './face_mask_detection/test/' + fname] for fname in test_image_order['filename']]


In [5]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
    print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
    print('Number of image names does not match the number of filepaths')

Number of image names i.e.  1536 matches the number of file paths i.e.  1536


In [7]:
# Converting the file_paths to dataframe
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,./face_mask_detection/test/Image_1.jpg
1,Image_2.jpg,./face_mask_detection/test/Image_2.jpg
2,Image_3.jpg,./face_mask_detection/test/Image_3.jpg
3,Image_4.jpg,./face_mask_detection/test/Image_4.jpg
4,Image_5.jpg,./face_mask_detection/test/Image_5.jpg


In [8]:
test_data = pd.merge(test_images, test_image_order, how = 'inner', on = 'filename')
test_data.head()   

,filename,filepaths,label
0,Image_1.jpg,./face_mask_detection/test/Image_1.jpg,NaN
1,Image_2.jpg,./face_mask_detection/test/Image_2.jpg,NaN
2,Image_3.jpg,./face_mask_detection/test/Image_3.jpg,NaN
3,Image_4.jpg,./face_mask_detection/test/Image_4.jpg,NaN
4,Image_5.jpg,./face_mask_detection/test/Image_5.jpg,NaN


In [9]:
test_data.drop('label', axis = 1, inplace=True)

In [10]:
test_data.head()

,filename,filepaths
0,Image_1.jpg,./face_mask_detection/test/Image_1.jpg
1,Image_2.jpg,./face_mask_detection/test/Image_2.jpg
2,Image_3.jpg,./face_mask_detection/test/Image_3.jpg
3,Image_4.jpg,./face_mask_detection/test/Image_4.jpg
4,Image_5.jpg,./face_mask_detection/test/Image_5.jpg


In [11]:
# Function for Resizing and Reshaping the input images
def centering_image(img):
    
    size = [256,256]
    #shape:(h×w×t)taking the first2 elements(h,w) and unpacking them appropriately 
    img_size = img.shape[:2]
    
    # extracting the excess space for centering.
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    
    #creating centered image by taking a 0-matrix and then re-assigning intensities
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized

In [12]:
# Image Resizing and converting them to array for test images of folder test
""" 
TQDM is a progress bar library. 
Inserting tqdm (or python -m tqdm)between pipes will pass 
through all stdin to stdout while printing progress to stderr
"""
tst_images = []
with tqdm(total=len(test_data)) as pbar:
    for i, file_path in enumerate(test_data.filepaths.values):
        
        #read image
        img = cv2.imread(file_path,1)
        
        #color order is changed
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        #resize
        #converting images to square
        if(img.shape[0] > img.shape[1]):
            tile_size = (int(img.shape[1]*256/img.shape[0]),256)
        else:
            tile_size = (256, int(img.shape[0]*256/img.shape[1]))

        #centering the images
        img = centering_image(cv2.resize(img, dsize=tile_size))

        #out put 224*224px 
        img = img[16:240, 16:240]
        tst_images.append(img)
        pbar.update(1)

tst_images = np.array(tst_images)
tst_images = tst_images/255 # Normalization

In [13]:
tst_images.shape


(1536, 224, 224, 3)

In [ ]:
#model_name = "ownModel_CNN_GPU.h5"
model = load_model('/content/drive/My Drive/dphi_bootcamp//model_MobileNetV2_transferLearning_TPU1_5epochs.h5')